In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data

/content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 전처리

## label_encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train['type'])
train['type'] = label_encoder.transform(train['type'])
test['type'] = label_encoder.transform(test['type'])

# 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
minmax_col_list = ['fixed acidity', 'citric acid' ,'chlorides','free sulfur dioxide','density','pH','sulphates']
robust_col_list = ['volatile acidity' ,'total sulfur dioxide', 'residual sugar','alcohol']

In [ ]:
min_max_scaler = MinMaxScaler()
scaled_train_data = min_max_scaler.fit_transform(train[minmax_col_list])
scaled_test_data = min_max_scaler.transform(test[minmax_col_list])

train[minmax_col_list] = scaled_train_data
test[minmax_col_list] = scaled_test_data

In [ ]:
robust_scaler = RobustScaler()
scaled_train_data = robust_scaler.fit_transform(train[robust_col_list])
scaled_test_data = robust_scaler.transform(test[robust_col_list])

train[robust_col_list] = scaled_train_data
test[robust_col_list] = scaled_test_data

# 모델링

In [ ]:
y = train_resampled['quality'] #퀄리티 예측해야할거
X = train_resampled.drop('quality', axis=1) #예측해야할거빼고 나머지

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [ ]:
model = lgb.LGBMClassifier()

stratified_split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

n_iter = 0
for train_idx, test_idx in stratified_split.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

     # 매개변수 그리드 정의
    param = {
      'num_leaves': [20, 30, 40],
      'learning_rate': [0.1, 0.01, 0.001],
      'n_estimators': [100, 500, 1000],
    }
    # GridSearchCV 객체 생성
    grid_search = GridSearchCV(model, param_grid=param, cv=5, refit=True, n_jobs=-1, return_train_score=True, verbose=2, error_score='raise')
    # 훈련 데이터로 모델 학습 및 최적 매개변수 탐색
    grid_search.fit(X_train, y_train)
    # 최적 매개변수 출력
    print("최적 매개변수:", grid_search.best_params_)
    # 최적 매개변수로 모델 재학습
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

y_pred = best_model.predict(test)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


# 제출파일

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission

In [ ]:
submission['quality'] = y_pred

In [ ]:
submission

In [ ]:
submission.to_csv('scale_lgbm.csv', index=False)